# Obliczenia dla robota mobilnego
- Współrzedne
- Ograniczenia poślizgu bocznego
- [ ] Ograniczenia poślizgu wzdłużego
- Macierz Pffafa

In [61]:
from sympy import *
# Zmienne i stale
l, l1, l2, l3 = symbols("l l_1 l_2 l_3", real=True)
a, a1, a2, a3 = symbols("a a_1 a_2 a_3", real=True)
t = symbols("t", real=True)

# Funkcje
xp = Function("x_p")(t)
yp = Function("y_p")(t)
yt = Function("y_t")(t)
xt = Function("x_t")(t)

phi     = Function("\\phi")(t)
theta   = Function("\\theta")(t)
xi      = Function("\\xi")(t)
dphi    = phi.diff(t).doit()
dtheta  = theta.diff(t).doit()
dxi     = xi.diff(t).doit()

# Skrocone postacie
ph, dph = phi, dphi
th, dth = theta, dtheta

________________________________
________________________________
## !!! Wprowadź wektor $q$


In [62]:
q = [ xp, yp, ph, th ]

## !!! Wprowadź (xp, yp) względem (xt, yt) lub odwrotnie:

In [63]:
# np.:
# xp = xt + l1 * cos(ph) + l2 * cos(ph+th)
# xs = None
# yp = yt + l1 * sin(ph) + l2 * sin(ph+th)
xt = xp - l1 * cos(ph) - l2 * cos(ph+th)
ys = None
yt = yp - l1 * sin(ph) - l2 * sin(ph+th)

## !!! Wprowadź orientacje osi

In [64]:
# np.:
orientacja_osi_P = ph + th
orientacja_osi_S = None
orientacja_osi_T = ph

________________________________
________________________________

## Współrzedne (x, y)
$$ x_p, y_p, x_t, y_t $$

In [65]:
xp

x_p(t)

In [66]:
yp

y_p(t)

In [67]:
xt

-l_1*cos(\phi(t)) - l_2*cos(\phi(t) + \theta(t)) + x_p(t)

In [68]:
yt

-l_1*sin(\phi(t)) - l_2*sin(\phi(t) + \theta(t)) + y_p(t)

## Pochodne współrzędnych
$$ [ \dot{x_p}, \dot{y_p}, \dot{x_t}, \dot{y_t} ]^T $$ 

In [69]:
dxp = xp.diff(t).doit()
dxp

Derivative(x_p(t), t)

In [70]:
dyp = yp.diff(t).doit()
dyp

Derivative(y_p(t), t)

In [71]:
dxt = xt.diff(t).doit()
dxt

l_1*sin(\phi(t))*Derivative(\phi(t), t) + l_2*(Derivative(\phi(t), t) + Derivative(\theta(t), t))*sin(\phi(t) + \theta(t)) + Derivative(x_p(t), t)

In [72]:
dyt = yt.diff(t).doit()
dyt

-l_1*cos(\phi(t))*Derivative(\phi(t), t) - l_2*(Derivative(\phi(t), t) + Derivative(\theta(t), t))*cos(\phi(t) + \theta(t)) + Derivative(y_p(t), t)

## Wektor $ \dot{q} $

In [73]:
dq = []
for q_i in q:
    dq.append( q_i.diff(t).doit() )
Matrix(dq)

Matrix([
[   Derivative(x_p(t), t)],
[   Derivative(y_p(t), t)],
[  Derivative(\phi(t), t)],
[Derivative(\theta(t), t)]])

In [74]:
BPW_P = None; BPW_S = None; BPW_T = None
BPB_P = None; BPB_S = None; BPB_T = None

## Obliczenia ograniczeń bocznych
### BPB_P - Brak poślizgu bocznego dla osi przedniej

In [75]:
BPB_P = dxp * sin( orientacja_osi_P ) - dyp * cos( orientacja_osi_P )
# BPB_P

In [76]:
BPB_P = BPB_P.simplify()
# BPB_P

In [77]:
BPB_P = collect(BPB_P, dq)   # Uporzatkowanie
BPB_P

sin(\phi(t) + \theta(t))*Derivative(x_p(t), t) - cos(\phi(t) + \theta(t))*Derivative(y_p(t), t)

### BPB_S - Brak poślizgu bocznego dla osi środkowej

In [79]:
if orientacja_osi_S != None:
    BPB_S = dxs * sin( orientacja_osi_S ) - dys * cos( orientacja_osi_S )

In [80]:
if orientacja_osi_S != None:
    BPB_S = BPB_S.simplify()

In [90]:
if orientacja_osi_S != None:
    BPB_S = collect(BPB_S, dq)
BPB_S

### BPB_T - Brak poślizgu bocznego dla osi tylniej

In [82]:
# BPB T - Brak poślizgu bocznego dla osi tylniej
BPB_T = dxt * sin( orientacja_osi_T ) - dyt * cos( orientacja_osi_T )
# BPB_T

In [83]:
BPB_T = BPB_T.simplify()
# BPB_T

In [84]:
BPB_T = collect(BPB_T, dq)   # Uporzatkowanie
BPB_T

l_2*cos(\theta(t))*Derivative(\theta(t), t) + (l_1 + l_2*cos(\theta(t)))*Derivative(\phi(t), t) + sin(\phi(t))*Derivative(x_p(t), t) - cos(\phi(t))*Derivative(y_p(t), t)

## Ograniczenia wzdłużne
### Oś przednia

In [85]:
BPW_P

### Oś środkowa

In [86]:
BPW_S

### Oś tylnia

In [87]:
BPW_T

## Macierz Pffafa

In [88]:
Aq = []
restrictions = [ BPB_P, BPB_S, BPB_T, BPW_P, BPW_S, BPW_T ]

for rest in restrictions:
    A_row = []
    if rest != None:
        for dq_i in dq:
            A_row.append( rest.coeff( dq_i ) )
        Aq.append(Matrix(A_row).T)

Matrix( Aq )

Matrix([
[sin(\phi(t) + \theta(t)), -cos(\phi(t) + \theta(t)),                        0,                  0],
[            sin(\phi(t)),             -cos(\phi(t)), l_1 + l_2*cos(\theta(t)), l_2*cos(\theta(t))]])

In [89]:
Matrix( dq )

Matrix([
[   Derivative(x_p(t), t)],
[   Derivative(y_p(t), t)],
[  Derivative(\phi(t), t)],
[Derivative(\theta(t), t)]])